In [1]:
from scipy.spatial.distance import pdist, squareform
from ripser import ripser
from itertools import product, starmap, islice
from ringity.classes import Dgm, DgmPt
from random import randint

import random
import scipy
import ringity
import numpy as np
import os
import time
import networkx as nx

In [2]:
RINGITY_PATH = os.path.dirname(ringity.__file__)

G = nx.read_edgelist(f"{RINGITY_PATH}/test_data/lipid_coexp_network.txt",
                    nodetype=int)
G = nx.convert_node_labels_to_integers(G)
dgm1 = ringity.core.diagram(G)

In [3]:
G[0]

AtlasView({1: {'net-flow': 15.097529008033787}, 2: {'net-flow': 24.933094332728327}, 3: {'net-flow': 29.792767289953545}, 4: {'net-flow': 20.423131648437703}, 5: {'net-flow': 23.993546332013082}, 6: {'net-flow': 23.872911276110855}, 7: {'net-flow': 13.075032163501263}, 8: {'net-flow': 47.38852257961669}, 9: {'net-flow': 24.193714517658833}, 10: {'net-flow': 36.470159760217996}, 11: {'net-flow': 23.153966556646694}, 12: {'net-flow': 37.9194247072103}, 13: {'net-flow': 37.52936306769482}, 14: {'net-flow': 35.5437923171357}, 15: {'net-flow': 24.448942435634372}, 16: {'net-flow': 23.934528235132237}, 17: {'net-flow': 23.330903751295576}, 18: {'net-flow': 27.26914523507974}, 19: {'net-flow': 22.93782100068431}, 20: {'net-flow': 26.995124889807144}, 21: {'net-flow': 26.824180232037683}, 22: {'net-flow': 24.79056820003825}, 23: {'net-flow': 37.47747084255292}, 24: {'net-flow': 41.79962463634473}, 25: {'net-flow': 59.25376182961267}, 26: {'net-flow': 33.942178334183254}, 27: {'net-flow': 42.79